In [1]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import cv2
import copy
import os
import numpy as np

def imgshow(img):
    if len(img.shape) == 3:
        plt.imshow(img[:, :, ::-1])
    else:
        plt.imshow(img,cmap='gray',vmin=0, vmax=255)
    plt.show() 


In [ ]:
def mkdir_always(path):
    if not os.path.isdir(path):
        os.mkdir(path)

In [ ]:
CHESSBOARD_SIZE = (6, 9)


def calibrate(chessboard_path, show_chessboard=False):
    # Logical coordinates of chessboard corners
    obj_p = np.zeros((1, CHESSBOARD_SIZE[0] * CHESSBOARD_SIZE[1], 3), np.float32)
    obj_p[0, :, :2] = np.mgrid[0:CHESSBOARD_SIZE[0], 0:CHESSBOARD_SIZE[1]].T.reshape(-1, 2)

    obj_points = []     # 3d point in real world space
    img_points = []     # 2d points in image plane.

    # Iterate through all images in the folder
    dirs = os.listdir(chessboard_path)
    image_list = [x for x in dirs if os.path.isfile(os.path.join(chessboard_path, x))]
    gray = None
    for image in image_list:
        img = cv2.imread(os.path.join(chessboard_path, image))
        #img = cv2.resize(img, (320,240), interpolation=cv2.INTER_AREA)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, CHESSBOARD_SIZE,
                                                 cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        if ret:
            # Refining corners position with sub-pixels based algorithm
            obj_points.append(obj_p)
            cv2.cornerSubPix(gray, corners, (3, 3), (-1, -1),
                             (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.01))
            img_points.append(corners)
            print('Image is valid for calibration:' + image )
            if show_chessboard:
                cv2.drawChessboardCorners(img, CHESSBOARD_SIZE, corners, ret)
                cv2.imwrite(os.path.join('./Chessboards_Corners', image), img)
        else:
            print('Image is NOT valid for calibration:' + image )

#     print(obj_points)  
#     print(img_points)   
            
    k = np.zeros((3, 3))
    d = np.zeros((4, 1))
    dims = gray.shape[::-1]
    num_valid_img = len(obj_points)
    if num_valid_img > 0:
        rvecs = [np.zeros((1, 1, 3), dtype=np.float64) for _ in range(num_valid_img)]
        tvecs = [np.zeros((1, 1, 3), dtype=np.float64) for _ in range(num_valid_img)]
        rms, _, _, _, _ = cv2.fisheye.calibrate(obj_points, img_points, gray.shape[::-1], k, d, rvecs, tvecs,
                                                # cv2.fisheye.CALIB_CHECK_COND +
                                                # When CALIB_CHECK_COND is set, the algorithm checks if the detected corners of each images are valid.
                                                # If not, an exception is thrown which indicates the zero-based index of the invalid image.
                                                # Such image should be replaced or removed from the calibration dataset to ensure a good calibration.
                                                cv2.fisheye.CALIB_RECOMPUTE_EXTRINSIC +
                                                cv2.fisheye.CALIB_FIX_SKEW +
                                                cv2.fisheye.CALIB_FIX_PRINCIPAL_POINT,
                                                (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6))
    print("Found " + str(num_valid_img) + " valid images for calibration")
    return k, d, dims



if not os.path.exists('./parameters'):
    os.makedirs('./parameters')
if not os.path.exists('./Chessboards_Corners'):
    os.makedirs('./Chessboards_Corners')

K, D, Dims = calibrate('./Chessboards', show_chessboard=True)
np.save('./parameters/Dims', np.array(Dims))
np.save('./parameters/K', np.array(K))
np.save('./parameters/D', np.array(D))

In [ ]:
def calc_unfish_map(k, d, dims,alpha=0.5,scale=1,fixed_point = True):  
    w=dims[0]
    h=dims[1]
    k[0,0]=k[0,0]*scale
    k[1,1]=k[1,1]*scale
    k[0,2]=(k[0,2]-(w-1)/2)*scale + (w*scale-1)/2
    k[1,2]=(k[1,2]-(h-1)/2)*scale + (h*scale-1)/2
    
    dims = (int(dims[0]*scale),int(dims[1]*scale))
    
    nk = k.copy()
   
    # #enlarge image twice to get corner info
    #nk[0,0]=k[0,0]/2
    #nk[1,1]=k[1,1]/2
    
#     nk[0,0]=k[0,0]/1.25
#     nk[1,1]=k[1,1]/1.25
    
#     nk, roi = cv2.getOptimalNewCameraMatrix(k, d, dims, 0,centerPrincipalPoint = True)

#     nk, roi = cv2.getOptimalNewCameraMatrix(k, d, dims, alpha, dims,centerPrincipalPoint = True)
    
#     print(k)
#     print(nk)

#     print(roi)
        
        
    map1, map2 = cv2.fisheye.initUndistortRectifyMap(k, d, np.eye(3), nk, dims, cv2.CV_32F)
    
    if fixed_point:
        map1,map2 = cv2.convertMaps(map1, map2, cv2.CV_16SC2)



    return map1,map2

In [ ]:
Dims = tuple(np.load('./parameters/Dims.npy'))
K = np.load('./parameters/K.npy')
D = np.load('./parameters/D.npy')

print(K)
print(D)
print(Dims)

w = Dims[0]
h = Dims[1]



unfishmap1,unfishmap2=calc_unfish_map(k=K, d=D, dims=Dims,alpha=0.6,scale=1.0)

f1,f2=calc_unfish_map(k=K, d=D, dims=Dims,alpha=0.6,scale=1.0,fixed_point=False)

small_unfishmap1,small_unfishmap2=calc_unfish_map(k=K, d=D, dims=Dims,alpha=0.6,scale=0.5)

# print(f1.shape)
# print(f2.shape)

# print(K)
#print(D)

# k_test = np.array( [[338.37324094,0,319.5],[0,339.059099,239.5],[0,0,1]],dtype=np.float64)
# D_test = np.array( [[ 0.01794191], [-0.12190366],[ 0.14111533],[-0.09602948]],dtype=np.float64)
# Dims_test = (640, 480)
# print(k_test)
# print(D_test)
# print(Dims_test)


In [ ]:
with open('unfishmap1.csv', 'wb') as f:
    np.savetxt(f, unfishmap1.reshape(-1, unfishmap1.shape[-1]), fmt='%d', delimiter=",",newline='\n')

with open('unfishmap2.csv', 'wb') as f:
    np.savetxt(f, unfishmap2, fmt='%d', delimiter=",",newline='\n')


In [ ]:
print(unfishmap1[0,0:10])
print(unfishmap1[0,-10::])

print(unfishmap1)


In [ ]:
# print(unfishmap1[::,::,0])

center_x = (w-1)/2
imgshow( abs(unfishmap1[::,::,0]-center_x)/center_x*255  )
center_y = (h-1)/2
imgshow( abs(unfishmap1[::,::,1]-center_y)/center_y*255  )

In [ ]:
x_curve = np.linspace(-center_x/1.25,center_x/1.25,640)+center_x

s = (h,w,2)

map1 = np.zeros(s,dtype=np.float32)
map1[::,::,0] = x_curve


img = abs(unfishmap1[::,::,0]-map1[::,::,0])

plt.imshow(img,cmap='gray')

print(img)


In [ ]:
imgs = [cv2.imread('../angle_screenshots/img_{}.png'.format(i*10)) for i in range(0,10+1)]



for img in imgs:
    imgshow(img)
    img = cv2.remap(img, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT) 
    imgshow(img)



In [ ]:
imgs = [cv2.imread('../angle_screenshots/img_{}.png'.format(i*10)) for i in range(0,10+1)]



for img in imgs:
    img = cv2.resize(img, (320,240), interpolation=cv2.INTER_AREA)
    imgshow(img)
    img = cv2.remap(img, small_unfishmap1, small_unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT) 
    imgshow(img)


In [ ]:
# img = cv2.imread('../angle_screenshots/img_50.png')
#img = cv2.imread('../angle_shots/img_055_040.png')
img = cv2.imread('Chessboards\img_021.jpg')
# imgshow(img)

p0 = (165,260)
p1 = (175,410)
points=[]
points.append(p0)
points.append(p1)

# print(points)

img2= img.copy()
cv2.circle(img2,tuple(points[0]),6,(0,0,255),2)
cv2.circle(img2,tuple(points[1]),6,(0,0,255),2)
imgshow(img2)

img2 = cv2.remap(img, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT) 

# p0b= (f2[p0[1],p0[0]],f1[p0[1],p0[0]])
# print(p0b)
# cv2.circle(img2,p0b,6,(0,0,255),2)
# imgshow(img2)

# points = np.zeros((len(points),1,2), dtype=np.float32)
# points[0][0] = p0
# points[1][0] = p1

# print(points)
# undistorted = cv2.undistortPoints(points,K,D)
# print(undistorted)


# cv2.circle(img2,(int(undistorted[0][0][0]),int(undistorted[0][0][1])),6,(0,0,255),2)
# cv2.circle(img2,(int(undistorted[1][0][0]),int(undistorted[1][0][1])),6,(0,0,255),2)
# imgshow(img2)



# def undistort(uv_orig):
#     # convert the point into the proper format for opencv
#     uv_raw = np.zeros((1,1,2), dtype=np.float32)
#     uv_raw[0][0] = (uv_orig[0], uv_orig[1])
#     # do the actual undistort
# #     nk, roi = cv2.getOptimalNewCameraMatrix(K, D, Dims, 0.6, Dims,centerPrincipalPoint = True)
#     nk = K
#     uv_new = cv2.undistortPoints(uv_raw, K, D, np.eye(3),nk)
#     # print(uv_orig, type(uv_new), uv_new)
#     return (int(uv_new[0][0][0]),int(uv_new[0][0][1]))


# p0b = tuple(undistort(p0))
# p1b = tuple(undistort(p1))
# # print(p0b)

# # p0b = (int(p0b[0]* 1.0+-10),int(p0b[1]* 1.0+30))
# # p1b = (int(p1b[0]* 1.0+-10),int(p1b[1]* 1.0+30))
    
# cv2.circle(img2,p0b,6,(0,0,255),2)
# cv2.circle(img2,p1b,6,(0,0,255),2)
# imgshow(img2)

point_matrix = np.zeros(shape=(len(points),1,2),dtype=np.float32)


# fill Pointfile in n x 1 x 2-Matrix
for i in range(0, len(points)):
    point_matrix[i][0][0] = points[i][0]
    point_matrix[i][0][1] = points[i][1]

# points2 = cv2.undistortPoints(point_matrix,K,D,R=np.eye(3),P=K)
# points2 = cv2.undistortPoints(point_matrix,K,D,R=None,P=K)
# points2 = cv2.undistortPoints(point_matrix,K,D)
# points2 = cv2.undistortPointsIter(point_matrix,K,D,None,K,criteria=(cv2.TERM_CRITERIA_MAX_ITER, 30, 1e-6))


print(points2)

# K_new = K
# fx = K_new[0, 0]
# fy = K_new[1, 1]
# cx = K_new[0, 2]
# cy = K_new[1, 2]

# point_matrix2 = np.zeros(shape=(len(points),2),dtype=np.int32)

# for i  in range(len(points2)):
#     px = points2[i][0][0]
#     py = points2[i][0][1]    
#     point_matrix2[i, 0] = px * fx + cx
#     point_matrix2[i, 1] = py * fy + cy

# print(point_matrix2)

# for p in point_matrix2:
# #     print(p)
# #     p = (int(p[0][0]*100)+320,int(p[0][1]*100)+240)
#     p = (p[0],p[1])
#     print(p)
#     cv2.circle(img2,p,6,(0,0,255),2)
# imgshow(img2)


for p in points2:
#     print(p)
#     p = (int(p[0][0]*100)+320,int(p[0][1]*100)+240)
    p = (int(p[0][0]),int(p[0][1]))
    print(p)
    cv2.circle(img2,p,6,(0,0,255),2)
imgshow(img2)
    
    
    
    
    

# img2 = cv2.remap(img, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT) 
# p3 = (195,255)
# cv2.circle(img2,p3,6,(0,0,255),2)
# imgshow(img2)

# img2= img.copy()
# p3b = tuple(undistort(p3))
# # print(p0b)
# cv2.circle(img2,p3b,6,(0,0,255),2)
# imgshow(img2)



# p0b = tuple(undistort(p0))
# # print(p0b)
# cv2.circle(img2,p0b,6,(0,0,255),2)
# imgshow(img2)

# print(unfishmap1)
# print(small_unfishmap1)
# print(f1)


# print(unfishmap1[p0[1],p0[0]])
# p0b = unfishmap1[p0[1],p0[0]]
# p0b = (p0b[0],p0b[1])
# cv2.circle(img2,p0b,6,(0,0,255),2)
# cv2.circle(img2,p0,6,(0,0,255),2)
# imgshow(img2)

# img3 = cv2.remap(img, f1, f2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT) 
# imgshow(img3)

# img2 = cv2.remap(img, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT) 
# imgshow(img2)


# print(f1)

# print(f2[190,310])
# p0b= (f2[p0[1],p0[0]],f1[p0[1],p0[0]])

# print(p0b)

# cv2.circle(img2,p0b,6,(0,0,255),2)
# imgshow(img2)


In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150
import cv2
import copy
import os
import numpy as np

def imgshow(img):
    if len(img.shape) == 3:
        plt.imshow(img[:, :, ::-1])
    else:
        plt.imshow(img,cmap='gray',vmin=0, vmax=255)
    plt.show() 


K = np.array( [[338.37324094,0,319.5],[0,339.059099,239.5],[0,0,1]],dtype=np.float64)
D = np.array( [[ 0.01794191], [-0.12190366],[ 0.14111533],[-0.09602948]],dtype=np.float64)
new_size = (640, 480)

Knew = K.copy()
# alpha = 0.6
# Knew, roi = cv2.getOptimalNewCameraMatrix(K, D, new_size, alpha, new_size,centerPrincipalPoint = True)
    
unfishmap1, unfishmap2 = cv2.fisheye.initUndistortRectifyMap(K, D, np.eye(3), Knew, new_size, cv2.CV_32F)
unfishmap1, unfishmap2 = cv2.convertMaps(unfishmap1, unfishmap2, cv2.CV_16SC2)

img = cv2.imread('3FYUT.jpg')
img_undistorted = cv2.remap(img, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT) 

points1=[]
points1.append((165,260))
points1.append((175,410))

print(points1)

img2= img.copy()
for p in points1:
    cv2.circle(img2,p,6,(0,0,255),2)
imgshow(img2)

point_matrix = np.zeros(shape=(len(points1),1,2),dtype=np.float32)

for i in range(0, len(points1)):
    point_matrix[i][0][0] = points1[i][0]
    point_matrix[i][0][1] = points1[i][1]
        
points_undistorted = cv2.fisheye.undistortPoints(point_matrix,K,D,P=Knew)

points2=[]
for p in points_undistorted:
    points2.append( (int(p[0][0]),int(p[0][1])) )

print("fisheye.undistortPoints:")    
print(points2)

img2= img_undistorted.copy()
for p in points2:
    cv2.circle(img2,p,6,(0,0,255),2)
imgshow(img2)

print("expected:")
points3=[]
points3.append((155,265))
points3.append((150,443))

print(points3)

img2= img_undistorted.copy()
for p in points3:
    cv2.circle(img2,p,6,(0,0,255),2)
imgshow(img2)
    




In [ ]:
# img1 = cv2.imread('../1d_angle_shots/img_060_040.png')
# # imgshow(img)

# img1 = cv2.remap(img1, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

# # kernel = np.ones((5,5),np.float32)/25
# # img = cv2.filter2D(img,-1,kernel)
# # img = cv2.blur(img,(5,5))

# gray1 = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

# corners1 = cv2.goodFeaturesToTrack(gray1,20,0.01,20)
# corners1 = np.int0(corners1)

# # print(corners)

# for i in corners1:
#     x,y = i.ravel()
#     cv2.circle(img1,(x,y),10,(0,255,0),2)
    
# # imgshow(img1)

# img2 = cv2.imread('../1d_angle_shots/img_065_040.png')
# # imgshow(img)
# img2 = cv2.remap(img2, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
# # img = cv2.filter2D(img,-1,kernel)
# # img = cv2.blur(img,(5,5))

# gray2 = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)

# corners2 = cv2.goodFeaturesToTrack(gray2,20,0.05,20)
# corners2 = np.int0(corners2)

# for i in corners2:
#     x,y = i.ravel()
#     cv2.circle(img2,(x,y),10,(0,255,0),2)
    
# # imgshow(img2)


# fmat = cv2.findFundamentalMat(corners1, corners2, cv2.FM_RANSAC, 10,0.99)
# # print(fmat)


# src1 = np.float32(gray1)             # convert first into float32
# src2 = np.float32(gray2)             # convert second into float32  
# pixel_shift, response = cv2.phaseCorrelate(src1,src2)
# # print(pixel_shift)


# # imgshow(gray1[::,0:100])

# src1 = np.float32(gray1[::,20:60])           
# src2 = np.float32(gray2[::,20:60])             
# pixel_shift, response = cv2.phaseCorrelate(src1,src2)
# # print(pixel_shift)


# # imgshow(gray1[50:-50,270:370])

# src1 = np.float32(gray1[50:-50,300:340])           
# src2 = np.float32(gray2[50:-50,300:340])             
# pixel_shift, response = cv2.phaseCorrelate(src1,src2)
# # print(pixel_shift)


ratio = 3
kernel_size = 3
def CannyThreshold(src, val):
    src_gray = cv2.cvtColor(src,cv2.COLOR_BGR2GRAY)
    low_threshold = val
    img_blur = cv2.blur(src_gray, (3,3))
    detected_edges = cv2.Canny(img_blur, low_threshold, low_threshold*ratio, kernel_size)
    mask = detected_edges != 0
    dst = src * (mask[:,:,None].astype(src.dtype))
    return dst

# gray1 = cv2.blur(gray1,(5,5))
# src1 = cv2.Sobel(gray1, cv2.CV_16S, dx=1, dy=0)
# imgshow(255-src1)

# gray2 = cv2.blur(gray2,(5,5)
# src2 = cv2.Sobel(gray2, cv2.CV_8U, dx=1, dy=0)
# imgshow(255-src2)

img1 = cv2.imread('../1d_angle_shots/img_000_040.png')
#img1 = cv2.remap(img1, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

img2 = cv2.imread('../1d_angle_shots/img_010_040.png')
# img2 = cv2.remap(img2, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

src1 = img1
src1 = CannyThreshold(src1,40)
src1 = cv2.remap(src1, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
src1 = cv2.cvtColor(src1,cv2.COLOR_BGR2GRAY)
imgshow(255-src1)
# imgshow(src1)


src2 = img2
src2 = CannyThreshold(src2,40)
src2 = cv2.remap(src2, unfishmap1, unfishmap2, interpolation=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)
src2 = cv2.cvtColor(src2,cv2.COLOR_BGR2GRAY)
imgshow(255-src2)
# imgshow(src2)

src1_f = np.float32(src1)             # convert first into float32
src2_f = np.float32(src2)             # convert second into float32  

pixel_shift, response = cv2.phaseCorrelate(src1_f,src2_f)
print(pixel_shift)


# square_size = (30,40)
# for x in range(img1.shape[1]//square_size[1]):
#     src1_s = src1_f [50:-50,x:(x+1)*square_size[1]-1]
#     src2_s = src2_f [50:-50,x:(x+1)*square_size[1]-1]
#     pixel_shift, response = cv2.phaseCorrelate(src1_s,src2_s)
#     print(pixel_shift)
# #     print(x*square_size[1])


square_size = (60,80)
for x in range(img1.shape[1]//square_size[1]):
    print(x)
    for y in range(img1.shape[0]//square_size[0]):
        src1_s = src1_f [y:(y+1)*square_size[0]-1,x:(x+1)*square_size[1]-1]
        src2_s = src2_f [y:(y+1)*square_size[0]-1,x:(x+1)*square_size[1]-1]
        pixel_shift, response = cv2.phaseCorrelate(src1_s,src2_s)

        print(pixel_shift)
    #     print(x*square_size[1])




# corners1 = cv2.goodFeaturesToTrack(src1,20,0.01,20)
# corners1 = np.int0(corners1)

# for i in corners1:
#     x,y = i.ravel()
#     cv2.circle(src1,(x,y),10,(255,255,255),2)

# imgshow(255-src1)


In [15]:
k = np.array( [[338.37324094,0,319.5],[0,339.059099,239.5],[0,0,1]],dtype=np.float64)
d = np.array( [[ 0.01794191], [-0.12190366],[ 0.14111533],[-0.09602948]],dtype=np.float64)
s = (640, 480)
b = 0.5

nk = cv2.fisheye.estimateNewCameraMatrixForUndistortRectify(k,d,s,np.eye(3),None,b);

# print(nk)

# [250, 300] -> [248.286, 301.492] -> [248.286, 301.492] -> [-53.9954, -214.952]

# original_point = (260.273, 289.896)
# # original_point = (319.2, 239.5)
# # original_point = (0, 0)



# # points_undistorted= np.array(  [[260.273, 289.896]],dtype=np.float64)

# points_2d = np.asarray([original_point])
# points_2d = points_2d[:, 0:2].astype('float32')
# points2d_undist = np.empty_like(points_2d)
# points_2d = np.expand_dims(points_2d, axis=1)

# points_undistorted = points_2d;

# point_redistorted = cv2.fisheye.distortPoints(points_undistorted,k,d);
        
# print(points_undistorted)
# print(point_redistorted)


# # original_point = (260.273, 289.896)
# # original_point = (319.5, 239.5)
# original_point = (309.5, 239.5)
# # original_point = (0,0)


# center_x = (s[0]-1)/2
# center_y = (s[1]-1)/2
# original_point = (original_point[0]-center_x,original_point[1]-center_y)

# # [250, 300] -> [260.273, 289.896] -> [260.273, 289.896] -> [-72.9805, -198.536]

# # points_undistorted= np.array(  [[260.273, 289.896]],dtype=np.float64)

# points_2d = np.asarray([original_point])
# points_2d = points_2d[:, 0:2].astype('float32')
# points2d_undist = np.empty_like(points_2d)
# points_2d = np.expand_dims(points_2d, axis=1)

# points_undistorted = points_2d;

# point_redistorted = cv2.fisheye.distortPoints(points_undistorted,nk,d);
        
# print(points_undistorted)
# print(point_redistorted)


# [250, 300] -> [268.386, 283.119] -> [268.386, 283.119] -> [269.015, 282.582]
original_point = (268.386, 283.119)
# original_point = (248.286, 301.492)
#original_point = (319.5, 239.5)
# original_point = (640, 239.5)
# original_point = (0,0)
#original_point = (639,479)



points_undistorted = np.array([original_point])

distorted_px = cv2.convertPointsToHomogeneous(points_undistorted)  
cam_intr_inv = np.linalg.inv(nk)
print(cam_intr_inv)
print(distorted_px)
distorted_px = np.tensordot(distorted_px, cam_intr_inv, axes=(2, 1))  
print(distorted_px)
distorted_px = cv2.convertPointsFromHomogeneous(distorted_px) 
a = 0
distorted_px = cv2.fisheye.distortPoints(distorted_px, k, d,None,a) 

# distorted_px = distorted_px.reshape((480, 640, 2))

point_redistorted = distorted_px

print(points_undistorted)
print(point_redistorted)


[[ 0.00418818  0.         -1.33450723]
 [ 0.          0.00417971 -1.00051899]
 [ 0.          0.          1.        ]]
[[[268.386 283.119   1.   ]]]
[[[-0.21045761  0.18283656  1.        ]]]
[[268.386 283.119]]
[[[250.0005756  300.00050645]]]


In [194]:
# distorted_px = cv2.convertPointsToHomogeneous(points_undistorted)  
# d = np.array((1,1,3),dtype=np.float64)
# for i in range(3):
#     for j in range(3):
#         d[i] += distorted_px[0,0,i] * cam_intr_inv[j,j]
# print(d)
                

[250.75204312 304.27220618   4.00590465]


In [24]:
print(k)

[[338.37324094   0.         319.5       ]
 [  0.         339.059099   239.5       ]
 [  0.           0.           1.        ]]


In [25]:
print(nk)

[[238.76703833   0.         318.6363392 ]
 [  0.         239.25100183 239.37517051]
 [  0.           0.           1.        ]]
